# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f62354729d0>
├── 'a' --> tensor([[ 0.4736, -0.7705,  0.4701],
│                   [ 1.1504, -0.0560,  0.7721]])
└── 'x' --> <FastTreeValue 0x7f6235472d00>
    └── 'c' --> tensor([[ 1.4792,  0.6494,  0.9899, -0.3853],
                        [-1.1993, -0.0255, -0.7803, -0.1843],
                        [-2.5944, -0.5409,  0.0834, -1.4254]])

In [4]:
t.a

tensor([[ 0.4736, -0.7705,  0.4701],
        [ 1.1504, -0.0560,  0.7721]])

In [5]:
%timeit t.a

68.1 ns ± 2.48 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f62354729d0>
├── 'a' --> tensor([[-3.0722,  0.4149,  0.1779],
│                   [ 1.3213, -1.4803, -1.8192]])
└── 'x' --> <FastTreeValue 0x7f6235472d00>
    └── 'c' --> tensor([[ 1.4792,  0.6494,  0.9899, -0.3853],
                        [-1.1993, -0.0255, -0.7803, -0.1843],
                        [-2.5944, -0.5409,  0.0834, -1.4254]])

In [7]:
%timeit t.a = new_value

67.8 ns ± 1.36 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4736, -0.7705,  0.4701],
               [ 1.1504, -0.0560,  0.7721]]),
    x: Batch(
           c: tensor([[ 1.4792,  0.6494,  0.9899, -0.3853],
                      [-1.1993, -0.0255, -0.7803, -0.1843],
                      [-2.5944, -0.5409,  0.0834, -1.4254]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4736, -0.7705,  0.4701],
        [ 1.1504, -0.0560,  0.7721]])

In [11]:
%timeit b.a

70.8 ns ± 0.938 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.0534, -0.0865,  0.0902],
               [-1.0882, -1.2013,  1.0959]]),
    x: Batch(
           c: tensor([[ 1.4792,  0.6494,  0.9899, -0.3853],
                      [-1.1993, -0.0255, -0.7803, -0.1843],
                      [-2.5944, -0.5409,  0.0834, -1.4254]]),
       ),
)

In [13]:
%timeit b.a = new_value

600 ns ± 8.54 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

919 ns ± 9.94 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.9 µs ± 218 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

266 µs ± 17.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

268 µs ± 14.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f62354728e0>
├── 'a' --> tensor([[[ 0.4736, -0.7705,  0.4701],
│                    [ 1.1504, -0.0560,  0.7721]],
│           
│                   [[ 0.4736, -0.7705,  0.4701],
│                    [ 1.1504, -0.0560,  0.7721]],
│           
│                   [[ 0.4736, -0.7705,  0.4701],
│                    [ 1.1504, -0.0560,  0.7721]],
│           
│                   [[ 0.4736, -0.7705,  0.4701],
│                    [ 1.1504, -0.0560,  0.7721]],
│           
│                   [[ 0.4736, -0.7705,  0.4701],
│                    [ 1.1504, -0.0560,  0.7721]],
│           
│                   [[ 0.4736, -0.7705,  0.4701],
│                    [ 1.1504, -0.0560,  0.7721]],
│           
│                   [[ 0.4736, -0.7705,  0.4701],
│                    [ 1.1504, -0.0560,  0.7721]],
│           
│                   [[ 0.4736, -0.7705,  0.4701],
│                    [ 1.1504, -0.0560,  0.7721]]])
└── 'x' --> <FastTreeValue 0x7f6235472d90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

43.9 µs ± 1.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f62354c9be0>
├── 'a' --> tensor([[ 0.4736, -0.7705,  0.4701],
│                   [ 1.1504, -0.0560,  0.7721],
│                   [ 0.4736, -0.7705,  0.4701],
│                   [ 1.1504, -0.0560,  0.7721],
│                   [ 0.4736, -0.7705,  0.4701],
│                   [ 1.1504, -0.0560,  0.7721],
│                   [ 0.4736, -0.7705,  0.4701],
│                   [ 1.1504, -0.0560,  0.7721],
│                   [ 0.4736, -0.7705,  0.4701],
│                   [ 1.1504, -0.0560,  0.7721],
│                   [ 0.4736, -0.7705,  0.4701],
│                   [ 1.1504, -0.0560,  0.7721],
│                   [ 0.4736, -0.7705,  0.4701],
│                   [ 1.1504, -0.0560,  0.7721],
│                   [ 0.4736, -0.7705,  0.4701],
│                   [ 1.1504, -0.0560,  0.7721]])
└── 'x' --> <FastTreeValue 0x7f6235472ee0>
    └── 'c' --> tensor([[ 1.4792,  0.6494,  0.9899, -0.3853],
                        [-1.1993, -0.0255, -0.7803, -0.1843],
                 

In [23]:
%timeit t_cat(trees)

40.5 µs ± 1.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

76.8 µs ± 1.4 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.4792,  0.6494,  0.9899, -0.3853],
                       [-1.1993, -0.0255, -0.7803, -0.1843],
                       [-2.5944, -0.5409,  0.0834, -1.4254]],
              
                      [[ 1.4792,  0.6494,  0.9899, -0.3853],
                       [-1.1993, -0.0255, -0.7803, -0.1843],
                       [-2.5944, -0.5409,  0.0834, -1.4254]],
              
                      [[ 1.4792,  0.6494,  0.9899, -0.3853],
                       [-1.1993, -0.0255, -0.7803, -0.1843],
                       [-2.5944, -0.5409,  0.0834, -1.4254]],
              
                      [[ 1.4792,  0.6494,  0.9899, -0.3853],
                       [-1.1993, -0.0255, -0.7803, -0.1843],
                       [-2.5944, -0.5409,  0.0834, -1.4254]],
              
                      [[ 1.4792,  0.6494,  0.9899, -0.3853],
                       [-1.1993, -0.0255, -0.7803, -0.1843],
                       [-2.5944, -0.5409,  0.0834, -1.4254]],

In [26]:
%timeit Batch.stack(batches)

139 µs ± 3.28 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.4792,  0.6494,  0.9899, -0.3853],
                      [-1.1993, -0.0255, -0.7803, -0.1843],
                      [-2.5944, -0.5409,  0.0834, -1.4254],
                      [ 1.4792,  0.6494,  0.9899, -0.3853],
                      [-1.1993, -0.0255, -0.7803, -0.1843],
                      [-2.5944, -0.5409,  0.0834, -1.4254],
                      [ 1.4792,  0.6494,  0.9899, -0.3853],
                      [-1.1993, -0.0255, -0.7803, -0.1843],
                      [-2.5944, -0.5409,  0.0834, -1.4254],
                      [ 1.4792,  0.6494,  0.9899, -0.3853],
                      [-1.1993, -0.0255, -0.7803, -0.1843],
                      [-2.5944, -0.5409,  0.0834, -1.4254],
                      [ 1.4792,  0.6494,  0.9899, -0.3853],
                      [-1.1993, -0.0255, -0.7803, -0.1843],
                      [-2.5944, -0.5409,  0.0834, -1.4254],
                      [ 1.4792,  0.6494,  0.9899, -0.3853],
                   

In [28]:
%timeit Batch.cat(batches)

223 µs ± 5.88 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

535 µs ± 13.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
